# **Tutorial** - BERTopic Best Practices

Through the nature of BERTopic, its modularity, many variations of the topic modeling technique is possible. However, during the development and through the usage of the package, a set of best practices have been developed that generally lead to great results.

The following are a number of steps, parameters, and settings that you can use that will generally improve the quality of the resulting topics. In other words, after going through the quick start and getting a feeling for the API these steps should get you to the next level of performance.

**NOTE:**
    Although these are called *best practices*, it does not necessarily mean that they work across all use cases perfectly. The underlying modular nature of BERTopic is meant to take different use cases into account. After going through these practices it is advised to fine-tune wherever necessary.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="20%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic
!pip install datasets
!pip install openai
import tqdm as notebook_tqdm
from tqdm.autonotebook import tqdm, trange
import pandas as pd
import numpy as np
import time as Time

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we will use a dataset containing abstracts and metadata from [ArXiv articles](https://huggingface.co/datasets/arxiv_dataset).


In [7]:
from datasets import load_dataset

'''
dataset = load_dataset("CShorten/ML-ArXiv-Papers")["train"]

# Extract abstracts to train on and corresponding titles
abstracts = dataset["abstract"]
titles = dataset["title"]
print(abstracts[1])
'''
#!git clone https://github.com/Cesar85Hol/papers-class-NLP.git
dataset=pd.read_csv('/content/papers-class-NLP/input_data/articles_2023.csv')
print(type(dataset['Abstract'][3]))
dataset['Abstract'][3]


<class 'str'>


"BACKGROUND: Discrete choice experiments (DCEs) are increasingly used in health state valuation studies. OBJECTIVE: This systematic review updates the progress and new findings of DCE studies in the health state valuation, covering the period since the review of June 2018 to November 2022. The review reports the methods that are currently being used in DCE studies to value health and study design characteristics, and, for the first time, reviews DCE health state valuation studies published in the Chinese language. METHODS: English language databases PubMed and Cochrane, and Chinese language databases Wanfang and CNKI were searched using the self-developed search terms. Health state valuation or methodology study papers were included if the study used DCE data to generate a value set for a preference-based measure. Key information extracted included DCE study design strategies applied, methods for anchoring the latent coefficient on to a 0-1 QALY scale and data analysis methods. RESULTS

In [8]:
# functions preparing the dataset for BERTopic architecture

tot_nan=dataset['Title'].isnull().sum()
print(f"total nan Titles in dataset: {tot_nan}")
tot_nan=dataset['Abstract'].isnull().sum()
print(f"total nan abstracts in dataset: {tot_nan}")

#1. Removing duplicates
print(f"initial size: {dataset.shape}")
dataset=dataset.drop_duplicates(subset=['Title'], keep ='first')
print(f"new size of items after removing duplicates: {dataset.shape}")

#2. Manage nan _hyp1: if both title and abstract nan -> delete; if just 1 nan, keep item and put an empty string (to allow tokenization)
for elem in dataset.index:
    if pd.isnull(dataset.at[elem, 'Title']) and pd.isnull(dataset.at[elem, 'Abstract']):
        dataset.drop(elem, inplace=True)
    elif pd.isnull(dataset.at[elem, 'Title']):
        dataset.at[elem, 'Title'] = ''
    elif pd.isnull(dataset.at[elem, 'Abstract']):
        dataset.at[elem, 'Abstract'] = ''

tot_nan=dataset['Title'].isnull().sum()
print(f"total nan Titles in dataset after manage: {tot_nan}")
tot_nan=dataset['Abstract'].isnull().sum()
print(f"total nan abstracts in dataset after manage: {tot_nan}")


total nan Titles in dataset: 0
total nan abstracts in dataset: 44
initial size: (1215, 17)
new size of items after removing duplicates: (1214, 17)
total nan Titles in dataset after manage: 0
total nan abstracts in dataset after manage: 0


In [10]:
dataset['Title_Abstract']=dataset['Title'].fillna(' ')+' ' +dataset['Abstract'].fillna(' ')
dataset['Title_Abstract'][3]


"Discrete Choice Experiments in Health State Valuation: A Systematic Review of Progress and New Trends. BACKGROUND: Discrete choice experiments (DCEs) are increasingly used in health state valuation studies. OBJECTIVE: This systematic review updates the progress and new findings of DCE studies in the health state valuation, covering the period since the review of June 2018 to November 2022. The review reports the methods that are currently being used in DCE studies to value health and study design characteristics, and, for the first time, reviews DCE health state valuation studies published in the Chinese language. METHODS: English language databases PubMed and Cochrane, and Chinese language databases Wanfang and CNKI were searched using the self-developed search terms. Health state valuation or methodology study papers were included if the study used DCE data to generate a value set for a preference-based measure. Key information extracted included DCE study design strategies applied,

**🔥 Tip - Sentence Splitter 🔥**
***
 Whenever you have large documents, you typically want to split them up into either paragraphs or sentences. A nice way to do so is by using NLTK's sentence splitter which is nothing more than:

```python
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [sent_tokenize(abstract) for abstract in abstracts]
sentences = [sentence for doc in sentences for sentence in doc]
```

***

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
sentences = [sent_tokenize(abstract) for abstract in abstracts]
sentences = [sentence for doc in sentences for sentence in doc]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cesaraugustoseminarioyrigoyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Best Practices**

With feedback from the community throughout the development of BERTopic and the core maintainer's personal experience, there are a number of best practices developed that generally lead to an improved topic model.

The goal of these best practices to quickly guide the user to what is commonly used to speed-up training, improve performance, explore alternatives, etc. Instead of having to search through many issues and discussions, an overview of best practices are discussed here.

To start off, it is important to have a general idea of the pipeline of BERTopic as it relates to many of these best practices.

BERTopic can be viewed as a sequence of steps to create its topic representations. There are five steps to this process:

![https://maartengr.github.io/BERTopic/algorithm/default.svg](https://maartengr.github.io/BERTopic/algorithm/default.svg)

The pipeline above implies significant modularity of BERTopic. Each step in this process was carefully selected such that they are all somewhat independent from one another.

As a result, we can adopt the pipeline to the current state-of-the-art with respect to each individual step:

 ![https://maartengr.github.io/BERTopic/algorithm/modularity.svg](https://maartengr.github.io/BERTopic/algorithm/modularity.svg)

## **Pre-calculate Embeddings**
After having created our data, namely `abstracts`, we can dive into the very first best practice, **pre-calculating embeddings**.

BERTopic works by converting documents into numerical values, called embeddings. This process can be very costly, especially if we want to iterate over parameters. Instead, we can calculate those embeddings once and feed them to BERTopic to skip calculating embeddings each time.

In [11]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
#embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding_model= SentenceTransformer("neuml/pubmedbert-base-embeddings")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
embeddings = embedding_model.encode(dataset['Title_Abstract'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [13]:
'''
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

y=dataset['Label']
#split in train - val - test : 60% - 20% - 20%
X_train,X_test,y_train,y_test=train_test_split(embeddings,y,test_size=0.2,random_state=42)
mlp_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
scaler = StandardScaler()
pipeline=Pipeline([('scaler',scaler),('mlp',mlp_model)])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
'''

Accuracy: 0.9711934156378601


## **Preventing Stochastic Behavior**
In BERTopic, we generally use a dimensionality reduction algorithm to reduce the size of the embeddings. This is done to prevent the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) to a certain degree.

As a default, this is done with [UMAP](https://github.com/lmcinnes/umap) which is an incredible algorithm for reducing dimensional space. However, by default, it shows stochastic behavior which creates different results each time you run it. To prevent that, we will need to set a `random_state` of the model before passing it to BERTopic.

As a result, we can now fully reproduce the results each time we run the model.

In [14]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

## **Controlling Number of Topics**
There is a parameter to control the number of topics, namely `nr_topics`. This parameter, however, merges topics **after** they have been created. It is a parameter that supports creating a fixed number of topics.

However, it is advised to control the number of topics through the cluster model which is by default HDBSCAN. HDBSCAN has a parameter, namely `min_topic_size` that indirectly controls the number of topics that will be created.

A higher `min_topic_size` will generate fewer topics and a lower `min_topic_size` will generate more topics.

Here, we will go with `min_topic_size=40` to get around XXX topics.

In [26]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

## **Improving Default Representation**
The default representation of topics is calculated through [c-TF-IDF](https://maartengr.github.io/BERTopic/algorithm/algorithm.html#5-topic-representation). However, c-TF-IDF is powered by the [CountVectorizer](https://maartengr.github.io/BERTopic/getting_started/vectorizers/vectorizers.html) which converts text into tokens. Using the CountVectorizer, we can do a number of things:

* Remove stopwords
* Ignore infrequent words
* Increase

In other words, we can preprocess the topic representations **after** documents are assigned to topics. This will not influence the clustering process in any way.

Here, we will ignore English stopwords and infrequent words. Moreover, by increasing the n-gram range we will consider topic representations that are made up of one or two words.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=1, ngram_range=(1, 2))

## **Additional Representations**
Previously, we have tuned the default representation but there are quite a number of [other topic representations](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html) in BERTopic that we can choose from. From [KeyBERTInspired](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#keybertinspired) and [PartOfSpeech](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#partofspeech), to [OpenAI's ChatGPT](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#chatgpt) and [open-source](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#langchain) alternatives, many representations are possible.

In BERTopic, you can model many different topic representations simultanously to test them out and get different perspectives of topic descriptions. This is called [multi-aspect](https://maartengr.github.io/BERTopic/getting_started/multiaspect/multiaspect.html) topic modeling.

Here, we will demonstrate a number of interesting and useful representations in BERTopic:

* KeyBERTInspired
  * A method that derives inspiration from how KeyBERT works
* PartOfSpeech
  * Using SpaCy's POS tagging to extract words
* MaximalMarginalRelevance
  * Diversify the topic words
* OpenAI
  * Use ChatGPT to label our topics


In [17]:
!pip install typing-extensions --upgrade

import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="sk-...")
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}

## **Training**
Now that we have a set of best practices, we can use them in our training loop. Here, several different representations, keywords and labels for our topics will be created. If you want to iterate over the topic model it is advised to use the pre-calculated embeddings as that significantly speeds up training.

In [27]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(dataset['Title_Abstract'].tolist(), embeddings)

2024-11-20 23:42:16,979 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-20 23:42:20,911 - BERTopic - Dimensionality - Completed ✓
2024-11-20 23:42:20,913 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-20 23:42:20,962 - BERTopic - Cluster - Completed ✓
2024-11-20 23:42:20,967 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-20 23:42:36,114 - BERTopic - Representation - Completed ✓


In [28]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,447,-1_patients_patient_treatment_preferences,"[patients, patient, treatment, preferences, st...","[patient preferences, patient preference, pref...","[preferences, decision, preference, cancer, an...","[patients, patient, treatment, preferences, st...",[Treatment preferences of patients with relaps...
1,0,139,0_treatment_patients_patient_preferences,"[treatment, patients, patient, preferences, di...","[patient preferences, patient preference, psor...","[patients, preferences, psoriasis, preference,...","[treatment, patients, patient, preferences, di...",[A discrete choice experiment on oral and inje...
2,1,114,1_patient_care_patients_decision,"[patient, care, patients, decision, making, de...","[patient preferences, shared decision, decisio...","[care, decision, making, decision making, pref...","[patient, care, patients, decision, making, pr...",[Patient perceptions of in-hospital laboratory...
3,2,66,2_patients_risk_aortic_valve,"[patients, risk, aortic, valve, patient, treat...","[aortic stenosis, aortic valve, valve replacem...","[patients, aortic, valve, vte, cardiovascular,...","[patients, risk, aortic, valve, patient, treat...",[Lifetime Management of Patients With Severe A...
4,3,60,3_breast_reconstruction_patients_patient,"[breast, reconstruction, patients, patient, wo...","[breast reconstruction, mastectomy, breast, br...","[breast, reconstruction, mastectomy, flap, eye...","[breast, reconstruction, patients, patient, wo...",[Patient Preferences for Postmastectomy Breast...
5,4,59,4_telehealth_covid_covid 19_care,"[telehealth, covid, covid 19, care, 19, teleme...","[telehealth visits, telehealth, telemedicine, ...","[telehealth, covid 19, telemedicine, visits, 1...","[telehealth, care, telemedicine, pandemic, vis...",[Telehealth Preferences Among Patients With Ad...
6,5,56,5_prostate_prostate cancer_treatment_cancer,"[prostate, prostate cancer, treatment, cancer,...","[advanced prostate, prostate cancer, prostate,...","[prostate, prostate cancer, men, bladder, ici,...","[prostate, treatment, cancer, patients, patien...","[Prostate Cancer, Version 4.2023, NCCN Clinica..."
7,6,37,6_abortion_pregnancy_women_care,"[abortion, pregnancy, women, care, birth, cesa...","[abortion care, medication abortion, abortion,...","[abortion, pregnancy, women, cesarean, labor, ...","[abortion, pregnancy, women, care, birth, cesa...",[What attributes of abortion care affect peopl...
8,7,35,7_pain_patients_knee_arthroplasty,"[pain, patients, knee, arthroplasty, patient, ...","[knee arthroplasty, arthroplasty, tka, rotator...","[knee, arthroplasty, anesthesia, joint, outcom...","[pain, patients, knee, arthroplasty, patient, ...",[Association of Open Reduction and Internal Fi...
9,8,32,8_diabetes_type diabetes_type_insulin,"[diabetes, type diabetes, type, insulin, prefe...","[people diabetes, patient preferences, diabete...","[type diabetes, preferences, foot, attributes,...","[diabetes, type, insulin, preferences, foot, p...","[Effectiveness, reach, uptake, and feasibility..."


To get all representations for a single topic, we simply run the following:

In [29]:
topic_model.get_topic(1, full=True)

{'Main': [('patient', 0.024704231181976427),
  ('care', 0.024479422396364435),
  ('patients', 0.020179342751973507),
  ('decision', 0.016215013093338462),
  ('making', 0.012674841916693493),
  ('decision making', 0.012663911512720978),
  ('preferences', 0.011829949900157356),
  ('hospital', 0.011820255687700032),
  ('sdm', 0.010203025661619328),
  ('study', 0.01013760521306264)],
 'KeyBERT': [('patient preferences', 0.57803005),
  ('shared decision', 0.45913517),
  ('decision making', 0.44686857),
  ('advance care', 0.42639962),
  ('care planning', 0.3857898),
  ('patient participation', 0.3521524),
  ('end life', 0.34552783),
  ('preferences', 0.33624455),
  ('decision', 0.26818216),
  ('making', 0.26277968)],
 'MMR': [('care', 0.024479422396364435),
  ('decision', 0.016215013093338462),
  ('making', 0.012674841916693493),
  ('decision making', 0.012663911512720978),
  ('preferences', 0.011829949900157356),
  ('sdm', 0.010203025661619328),
  ('end life', 0.00853351136775783),
  ('comm

**NOTE**: The labels generated by OpenAI's **ChatGPT** are especially interesting to use throughout your model. Below, we will go into more detail how to set that as a custom label.

**🔥 Tip - Parameters 🔥**
***
If you would like to return the topic-document probability matrix, then it is advised to use `calculate_probabilities=True`. Do note that this can significantly slow down training. To speed it up, use [cuML's HDBSCAN](https://maartengr.github.io/BERTopic/getting_started/clustering/clustering.html#cuml-hdbscan) instead. You could also approximate the topic-document probability matrix with `.approximate_distribution` which will be discussed later.
***

## **(Custom) Labels**
The default label of each topic are the top 3 words in each topic combined with an underscore between them.

This, of course, might not be the best label that you can think of for a certain topic. Instead, we can use `.set_topic_labels` to manually label all or certain topics.

We can also use `.set_topic_labels` to use one of the other topic representations that we had before, like `KeyBERTInspired` or even `OpenAI`.

In [ ]:
# Label the topics yourself
topic_model.set_topic_labels({1: "Space Travel", 7: "Religion"})

# or use one of the other topic representations, like KeyBERTInspired
keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
topic_model.set_topic_labels(keybert_topic_labels)

# or ChatGPT's labels
chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
chatgpt_topic_labels[-1] = "Outlier Topic"
topic_model.set_topic_labels(chatgpt_topic_labels)

Now that we have set the updated topic labels, we can access them with the many functions used throughout BERTopic. Most notably, you can show the updated labels in visualizations with the `custom_labels=True` parameters.

In [ ]:
topic_model.get_topic_info()

Notice that the overview in `.get_topic_info` now also includes the column `CustomName`. That is the custom label that we just created for each topic.

## **Topic-Document Distribution**
If using `calculate_probabilities=True` is not possible, than you can [approximate the topic-document distributions](https://maartengr.github.io/BERTopic/getting_started/distribution/distribution.html) using `.approximate_distribution`. It is a fast and flexible method for creating different topic-document distributions.

In [30]:
# `topic_distr` contains the distribution of topics in each document
topic_distr, _ = topic_model.approximate_distribution(dataset['Title_Abstract'].tolist(), window=8, stride=4)

100%|██████████| 2/2 [00:02<00:00,  1.32s/it]


Next, lets take a look at a specific abstract and see how the topic distribution was extracted:

In [31]:
abstract_id = 10
print(dataset['Title_Abstract'].tolist()[abstract_id])

Linking Patients' Goals and Priorities to Recommendations for Medication Changes in a Polypharmacy-Focused Structured Clinical Pathway. Polypharmacy is associated with poorer health outcomes in older adults. It is challenging to minimize the harmful effects of medications while maximizing benefits of single-disease-focused recommendations. Integrating patient input can balance these factors. The objectives are to describe the goals, priorities, and preferences of participants asked about these in a structured process to polypharmacy, and to describe the extent that decision-making within the process mapped onto these, signaling a patient-centered approach. This is a single-group quasi-experimental study, nested within a feasibility randomized controlled trial. Patient goals and priorities were mapped to medication recommendations made during the intervention. Overall, there were 33 participants who reported 55 functional goals and 66 symptom priorities, and 16 participants reported unw

In [32]:
# Visualize the topic-document distribution for a single document
topic_model.visualize_distribution(topic_distr[abstract_id])

In [33]:
# Visualize the topic-document distribution for a single document
topic_model.visualize_distribution(topic_distr[abstract_id], custom_labels=True)

It seems to have extracted a number of topics that are relevant and shows the distributions of these topics across the abstract. We can go one step further and visualize them on a token-level:

In [34]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(dataset['Title_Abstract'].tolist()[abstract_id], calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(dataset['Title_Abstract'].tolist()[abstract_id], topic_token_distr[0])
df

100%|██████████| 1/1 [00:00<00:00, 88.89it/s]


,Linking,Patients,Goals,and,Priorities,to,Recommendations,for,Medication,Changes,in,Polypharmacy,Focused,Structured,Clinical,Pathway,Polypharmacy,is,associated,with,poorer,health,outcomes,in,older,adults,It,is,challenging,to,minimize,the,harmful,effects,of,medications,while,maximizing,benefits,of,single,disease,focused,recommendations,Integrating,patient,input,can,balance,these,factors,The,objectives,are,to,describe,the,goals,priorities,and,preferences,of,participants,asked,about,these,in,structured,process,to,polypharmacy,and,to,describe,the,extent,that,decision,making,within,the,process,mapped,onto,these,signaling,patient,centered,approach,This,is,single,group,quasi,experimental,study,nested,within,feasibility,randomized,controlled,trial,Patient,goals,and,priorities,were,mapped,to,medication,recommendations,made,during,the,intervention,Overall,there,were,33,participants,who,reported,55,functional,goals,and,66,symptom,priorities,and,16,participants,reported,unwanted,medications,Overall,154,recommendations,for,medication,alterations,occurred,Of,those,68,44,recommendations,mapped,to,the,individual,goals,and,priorities,whereas,the,rest,were,based,on,clinical,judgment,where,no,priorities,were,expressed,Our,results,signal,this,process,supports,patient,centered,approach,allowing,conversations,around,goals,and,priorities,in,structured,process,to,polypharmacy,should,be,integrated,into,subsequent,medication,decisions
1_patient_care_patients_decision,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.114,0.351,0.589,0.689,0.575,0.338,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


**🔥 Tip - `use_embedding_model` 🔥**
***
As a default, we compare the c-TF-IDF calculations between the token sets and all topics. Due to its bag-of-word representation, this is quite fast. However, you might want to use the selected embedding_model instead to do this comparison. Do note that due to the many token sets, it is often computationally quite a bit slower:

```python
topic_distr, _ = topic_model.approximate_distribution(docs, use_embedding_model=True)
```
***




## **Outlier Reduction**
By default, HDBSCAN generates outliers which is a helpful mechanic in creating accurate topic representations. However, you might want to assign every single document to a topic. We can use `.reduce_outliers` to map some or all outliers to a topic:

In [35]:
# Reduce outliers
new_topics = topic_model.reduce_outliers(dataset['Title_Abstract'].tolist(), topics)

# Reduce outliers with pre-calculate embeddings instead
new_topics = topic_model.reduce_outliers(dataset['Title_Abstract'].tolist(), topics, strategy="embeddings", embeddings=embeddings)

100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


**💡  NOTE - Update Topics with Outlier Reduction 💡**
***
After having generated updated topic assignments, we can pass them to BERTopic in order to update the topic representations:

```python
topic_model.update_topics(docs, topics=new_topics)
```

It is important to realize that updating the topics this way may lead to errors if topic reduction or topic merging techniques are used afterwards. The reason for this is that when you assign a -1 document to topic 1 and another -1 document to topic 2, it is unclear how you map the -1 documents. Is it matched to topic 1 or 2.
***

## **Visualize Topics**

With visualizations, we are closing into the realm of subjective "best practices". These are things that I generally do because I like the representations but your experience might differ.

Having said that, there are two visualizations that are my go-to when visualizing the topics themselves:

* `topic_model.visualize_topics()`
* `topic_model.visualize_hierarchy()`

In [36]:
topic_model.visualize_topics(custom_labels=True)

In [37]:
topic_model.visualize_hierarchy(custom_labels=True)

## **Visualize Documents**

When visualizing documents, it helps to have embedded the documents beforehand to speed up computation. Fortunately, we have already done that as a "best practice".

Visualizing documents in 2-dimensional space helps in understanding the underlying structure of the documents and topics.

In [38]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

The following plot is **interactive** which means that you can zoom in, double click on a label to only see that one and generally interact with the plot:

In [40]:
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
topic_model.visualize_documents(dataset['Title'].tolist(), reduced_embeddings=reduced_embeddings, custom_labels=True)

In [41]:
# We can also hide the annotation to have a more clear overview of the topics
topic_model.visualize_documents(dataset['Title'].tolist(), reduced_embeddings=reduced_embeddings, custom_labels=True, hide_annotations=True)

**💡  NOTE - 2-dimensional space 💡**
***
Although visualizing the documents in 2-dimensional gives an idea of their underlying structure, there is a risk involved.

Visualizing the documents in 2-dimensional space means that we have lost significant information since the original embeddings were more than 384 dimensions. Condensing all that information in 2 dimensions is simply not possible. In other words, it is merely an **approximation**, albeit quite an accurate one.
***

## **Serialization**

When saving a BERTopic model, there are several ways in doing so. You can either save the entire model with `pickle`, `pytorch`, or `safetensors`.

Personally, I would advise going with `safetensors` whenever possible. The reason for this is that the format allows for a very small topic model to be saved and shared.

When saving a model with `safetensors`, it skips over saving the dimensionality reduction and clustering models. The `.transform` function will still work without these models but instead assign topics based on the similarity between document embeddings and the topic embeddings.

As a result, the `.transform` step might give different results but it is generally worth it considering the smaller and significantly faster model.

In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("my_model_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

**💡  NOTE - Embedding Model 💡**
***
Using `safetensors`, we are not saving the underlying embedding model but merely a pointer to the model. For example, in the above example we are saving the string `"sentence-transformers/all-MiniLM-L6-v2"` so that we can load in the embedding model alongside the topic model.

This currently only works if you are using a sentence transformer model. If you are using a different model, you can load it in when loading the topic model like this:

```python
from sentence_transformers import SentenceTransformer

# Define embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load model and add embedding model
loaded_model = BERTopic.load("path/to/my/model_dir", embedding_model=embedding_model)
```
***

As mentioned above, loading can be done as follows:

In [ ]:
from sentence_transformers import SentenceTransformer

# Define embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load model and add embedding model
loaded_model = BERTopic.load("my_model_dir", embedding_model=embedding_model)

## **Inference**

To speed up the inference, we can leverage a "best practice" that we used before, namely serialization. When you save a model as `safetensors` and then load it in, we are removing the dimensionality reduction and clustering steps from the pipeline.

Instead, the assignment of topics is done through cosine similarity of document embeddings and topic embeddings. This speeds up inferences significantly.

To show its effect, let's start by disabling the logger:

In [ ]:
from bertopic._utils import MyLogger
logger = MyLogger("ERROR")
loaded_model.verbose = False
topic_model.verbose = False

Then, we run inference on both the loaded model and the non-loaded model:

In [ ]:
%timeit loaded_model.transform(abstracts[:100])

In [ ]:
%timeit topic_model.transform(abstracts[:100])

**1000 documents**

In [ ]:
%timeit loaded_model.transform(abstracts[:1000])

In [ ]:
%timeit topic_model.transform(abstracts[:1000])

**10_000 documents**

In [ ]:
%timeit loaded_model.transform(abstracts[:10000])

In [ ]:
%timeit topic_model.transform(abstracts[:10000])

Based on the above, the `loaded_model` seems to be quite a bit faster for inference than the original `topic_model`.